In [1]:
import numpy as np
import pandas as pd
from confluent_kafka import Producer
from connect import get_connection
from random import randint
from simulation import transaktion_factory
from time import sleep
import socket


from confluent_kafka import Consumer

# Einlesen des Buchbestands

In [2]:
df_bestand = pd.read_csv('data/Bestand.csv', sep=';', encoding='unicode_escape')

In [3]:
bestand_cols=df_bestand.columns

# Einlesen der Tabellen des Producers

In [4]:
c = Consumer({
    'bootstrap.servers': '0.0.0.0:9092',
    'group.id': socket.gethostname(),
    'auto.offset.reset': 'latest'
})


In [5]:
prod_Transaktion_cols={
    0:'Aktion',
    1:'Datum',
    2:'Fernleihe',
    3:'ID_Exemplar',
    4:'ID_Kunde',
    5:'Titel',
    6:'Autor',
    7:'Jahr',
    8:'Art',
    9:'Kennung',
   10:'Zugriffsort'
}

prod_Bewertung_cols={
    0:'Aktion',
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

prod_Neukunden_cols={
    0:'Aktion',
    1:'ID_Kunde',
    2:'c_count',
    3:'Kundennr',
    4:'Vorname_1',
    5:'Vorname_2',
    6:'Nachname',
    7:'Anrede',
    8:'PLZ',
    9:'Strasse',
   10:'Hausnr',
   11:'Mail',
   12:'Tel',
   13:'Geschlecht',
   14:'Geburtsdatum',
   15:'Beruf',
   16:'Titel',
   17:'PersoNr',
   18:'PersoValidTo' 
}

cols=[prod_Transaktion_cols,prod_Bewertung_cols,prod_Neukunden_cols]
topics=["Transaktion", "Bewertung", "Neukunden"]

dfd={}

In [6]:
for col, topic in zip(cols, topics):
    print(col, topic)

    dfd[topic] = pd.DataFrame(columns=list(col.values()))
    
    
    c.subscribe([topic])
    
    for i in range(60):
        msg = c.poll(1)
        if msg is None:
            continue
    
        mrow = pd.DataFrame([msg.value().decode('utf-8').split(";")]).rename(columns=col)

        dfd[topic]=pd.concat([dfd[topic], mrow])

    dfd[topic] = dfd[topic].reset_index(drop=True)
    #c.close()

{0: 'Aktion', 1: 'Datum', 2: 'Fernleihe', 3: 'ID_Exemplar', 4: 'ID_Kunde', 5: 'Titel', 6: 'Autor', 7: 'Jahr', 8: 'Art', 9: 'Kennung', 10: 'Zugriffsort'} Transaktion
{0: 'Aktion', 1: 'ID_Kunde', 2: 'ID_Buch', 3: 'Wertung', 4: 'Rezension'} Bewertung
{0: 'Aktion', 1: 'ID_Kunde', 2: 'c_count', 3: 'Kundennr', 4: 'Vorname_1', 5: 'Vorname_2', 6: 'Nachname', 7: 'Anrede', 8: 'PLZ', 9: 'Strasse', 10: 'Hausnr', 11: 'Mail', 12: 'Tel', 13: 'Geschlecht', 14: 'Geburtsdatum', 15: 'Beruf', 16: 'Titel', 17: 'PersoNr', 18: 'PersoValidTo'} Neukunden


In [16]:
dfd['Transaktion']

,Aktion,Datum,Fernleihe,ID_Exemplar,ID_Kunde,Titel,Autor,Jahr,Art,Kennung,Zugriffsort
0,Rückgabe,2024-10-18 23:00:00,False,98,750,Das Puschkinhaus,Andrej Bitow,1978,Roman,LXU03651,EOM82015
1,Leihe,2024-10-19 05:00:00,False,139,6282,Tod in Hollywood,Evelyn Waugh,1948,Roman,Rbt96075,Cra78145
2,Leihe,2024-10-19 07:00:00,False,243,5099,Die Jalousie oder die Eifersucht,Alain Robbe-Grillet,1957,Roman,jWG14352,cht96521
3,Rückgabe,2024-10-19 12:00:00,False,101,3066,Niels Lyhne,Jens Peter Jacobsen,1880,Roman,KXd04961,dUk72451
4,Leihe,2024-10-19 14:00:00,False,224,4066,Die Verlobten,Alessandro Manzoni,1827,Roman,sJB54392,cPX39872
5,Rückgabe,2024-10-19 19:00:00,False,111,6798,Nathan der Weise,Gotthold Ephraim Lessing,1797,Drama,mtg19253,zkV51872
6,Leihe,2024-10-19 21:00:00,False,70,424,Die New-York-Trilogie,Paul Auster,1987,Roman,BTz95430,Wpk86341
7,Rückgabe,2024-10-20 01:00:00,False,235,245,Rolandslied,Unbekannt,1100,Epos,GNH29501,wnP32974
8,Leihe,2024-10-20 03:00:00,False,167,3715,Unfrisierte Gedanken,Stanislaw Jerzy Lec,1957,Lyrik,zhG62495,qKh94861
9,Leihe,2024-10-20 05:00:00,False,258,3115,Die Klavierspielerin,Elfriede Jelinek,1983,Roman,HxQ98370,wnJ90864


In [8]:
dfd['Bewertung']

,Aktion,ID_Kunde,ID_Buch,Wertung,Rezension
0,Bewertung,70,787,6,Ich mochte die Grundidee und einige der Chara...
1,Bewertung,166,103,5,"Ein Buch, das in der Mitte schwebt – weder be..."
2,Bewertung,123,989,3,Verfehlt die Erwartungen.
3,Bewertung,214,171,5,Das Buch bietet eine durchschnittliche Erzähl...
4,Bewertung,172,89,6,"Das Buch hat seine Stärken, vor allem in der ..."


In [21]:
dfd['Neukunden']

,Aktion,ID_Kunde,c_count,Kundennr,Vorname_1,Vorname_2,Nachname,Anrede,PLZ,Strasse,Hausnr,Mail,Tel,Geschlecht,Geburtsdatum,Beruf,Titel,PersoNr,PersoValidTo
0,Neukunde,15,267,2510,Amelie,NaN,Fischer,Frau,97795,Wilhelmstraße,8,amelie.fischer@gmail.com,09747/443823,Weiblich,1937-02-07,Senior/in,NaN,SIJ14967,2023-11-29
1,Neukunde,16,268,7215,Sophie,NaN,Schmitz,Frau,26529,Neuer,9,sophie.schmitz@t-online.de,04936/218824,Weiblich,1959-01-11,Key Account Manager/in,NaN,NFB62714,2022-11-23
2,Neukunde,17,269,2706,Leonie,NaN,Meyer,Frau,24811,Königsberger,7,leonie.meyer@t-online.de,04336/83394,Weiblich,1936-10-22,Senior/in,NaN,DKN91743,2022-03-02


# Cleaning der Tabellen

### Transaktion

In [18]:
# IDs zu int konvertieren
dfd['Transaktion']['ID_Exemplar'] = pd.to_numeric(dfd['Transaktion']['ID_Exemplar']).astype(int)
dfd['Transaktion']['ID_Kunde'] = pd.to_numeric(dfd['Transaktion']['ID_Kunde']).astype(int)
# Jahre zu int konvertieren
dfd['Transaktion']['Jahr'] = pd.to_numeric(dfd['Transaktion']['Jahr']).astype(int)

# Fernleihe zu true oder false konvertieren
dfd['Transaktion']['Fernleihe'] = np.where(dfd['Transaktion']['Fernleihe'].astype(str).str.contains("None"), False, dfd['Transaktion']['Fernleihe'] )
dfd['Transaktion']['Fernleihe'] = np.where(dfd['Transaktion']['Fernleihe'].astype(str).str.contains("False"), False, dfd['Transaktion']['Fernleihe'] )
dfd['Transaktion']['Fernleihe'] = np.where(dfd['Transaktion']['Fernleihe'].astype(str).str.contains("True"), True, dfd['Transaktion']['Fernleihe'] )


### Bewertung

In [19]:
# IDs zu int konvertieren
dfd['Bewertung']['ID_Kunde'] = pd.to_numeric(dfd['Bewertung']['ID_Kunde']).astype(int)
dfd['Bewertung']['ID_Buch'] = pd.to_numeric(dfd['Bewertung']['ID_Buch']).astype(int)
dfd['Bewertung']['Wertung'] = pd.to_numeric(dfd['Bewertung']['Wertung']).astype(int)

### Neukunden

In [25]:
# IDs zu int konvertieren
dfd['Neukunden']['ID_Kunde'] = pd.to_numeric(dfd['Neukunden']['ID_Kunde']).astype(int)
dfd['Neukunden']['Kundennr'] = pd.to_numeric(dfd['Neukunden']['Kundennr']).astype(int)

# Anreden konsistent machen 

dfd['Neukunden']['Anrede'] = dfd['Neukunden']['Anrede'].replace(['Fr\.'],'Frau', regex=True)
dfd['Neukunden']['Anrede'] = dfd['Neukunden']['Anrede'].replace(['Hr\.'],'Herr', regex=True)
# Titel: "None" to NaN
dfd['Neukunden']['Titel']  = np.where(dfd['Neukunden']['Titel'].str.contains("None"), np.nan, dfd['Neukunden']['Titel'] )
# Vorname_2: "nan" to NaN
dfd['Neukunden']['Vorname_2']  = np.where(dfd['Neukunden']['Vorname_2'].str.contains("nan"), np.nan, dfd['Neukunden']['Vorname_2'] )
# Hausnr: "nan" to NaN
dfd['Neukunden']['Hausnr'] = dfd['Neukunden']['Hausnr'].astype(str)
dfd['Neukunden']['Hausnr']  = np.where(dfd['Neukunden']['Hausnr'].str.contains("nan"), np.nan, dfd['Neukunden']['Hausnr'] )
# 4-stellige PLZ mit "0" präfixen 
dfd['Neukunden']['PLZ']    = dfd['Neukunden']['PLZ'].str.strip().str.rjust(5, '0')

# Hausnummern aus Strasse-Spalte extrahieren
def extract_hausnr(strasse):
  if pd.isna(strasse):
    return np.nan
  parts = strasse.split()
  if len(parts) > 1 and parts[-1].isdigit():
    return parts[-1]
  else:
    return np.nan
#Apply extract_hausnr, wenn 'Hausnr' NaN und entferne die Hausnummern aus der Strasse Spalte
dfd['Neukunden'].loc[dfd['Neukunden']['Hausnr'].isnull(), 'Hausnr'] = dfd['Neukunden'].loc[dfd['Neukunden']['Hausnr'].isnull(), 'Strasse'].apply(extract_hausnr)
dfd['Neukunden']['Strasse'] = dfd['Neukunden']['Strasse'].str.rsplit(expand=True)[0]
# Convert Hausnr to int
dfd['Neukunden']['Hausnr'] = pd.to_numeric(dfd['Neukunden']['Hausnr']).astype(int)

# Geschlecht konsistent machen 
dfd['Neukunden']['Geschlecht'] = dfd['Neukunden']['Geschlecht'].str.strip().replace(["^w$", "^W$"],'Weiblich', regex=True)
dfd['Neukunden']['Geschlecht'] = dfd['Neukunden']['Geschlecht'].str.strip().replace(["^m$", "^M$"],'Männlich', regex=True)

# Datumsspalten konsistent machen
dfd['Neukunden']['Geburtsdatum'] = pd.to_datetime(dfd['Neukunden']['Geburtsdatum'],format='mixed', dayfirst=True)
dfd['Neukunden']['PersoValidTo'] = pd.to_datetime(dfd['Neukunden']['PersoValidTo'],format='mixed', dayfirst=True)

### Bestand

In [28]:
# Jahre zu int konvertieren
df_bestand['Jahr'] = pd.to_numeric(df_bestand['Jahr']).astype('Int64', errors='ignore')

In [29]:
dfd['Transaktion']

,Aktion,Datum,Fernleihe,ID_Exemplar,ID_Kunde,Titel,Autor,Jahr,Art,Kennung,Zugriffsort
0,Rückgabe,2024-10-18 23:00:00,False,98,750,Das Puschkinhaus,Andrej Bitow,1978,Roman,LXU03651,EOM82015
1,Leihe,2024-10-19 05:00:00,False,139,6282,Tod in Hollywood,Evelyn Waugh,1948,Roman,Rbt96075,Cra78145
2,Leihe,2024-10-19 07:00:00,False,243,5099,Die Jalousie oder die Eifersucht,Alain Robbe-Grillet,1957,Roman,jWG14352,cht96521
3,Rückgabe,2024-10-19 12:00:00,False,101,3066,Niels Lyhne,Jens Peter Jacobsen,1880,Roman,KXd04961,dUk72451
4,Leihe,2024-10-19 14:00:00,False,224,4066,Die Verlobten,Alessandro Manzoni,1827,Roman,sJB54392,cPX39872
5,Rückgabe,2024-10-19 19:00:00,False,111,6798,Nathan der Weise,Gotthold Ephraim Lessing,1797,Drama,mtg19253,zkV51872
6,Leihe,2024-10-19 21:00:00,False,70,424,Die New-York-Trilogie,Paul Auster,1987,Roman,BTz95430,Wpk86341
7,Rückgabe,2024-10-20 01:00:00,False,235,245,Rolandslied,Unbekannt,1100,Epos,GNH29501,wnP32974
8,Leihe,2024-10-20 03:00:00,False,167,3715,Unfrisierte Gedanken,Stanislaw Jerzy Lec,1957,Lyrik,zhG62495,qKh94861
9,Leihe,2024-10-20 05:00:00,False,258,3115,Die Klavierspielerin,Elfriede Jelinek,1983,Roman,HxQ98370,wnJ90864


# Aufbau der Tabellen für das Core DWH

In [30]:
dfs={}

In [31]:
Kunde_cols={
    0:'ID_Kunde',
    1:'ID_Mitgliedsstatus',
    2:'Kundennr',
    3:'Vorname_1',
    4:'Vorname_2',
    5:'Nachname',
    6:'Anrede',
    7:'Titel',
    8:'PLZ',
    9:'Strasse',
   10:'Hausnr',
   11:'Mail',
   12:'Tel',
   13:'Geschlecht',
   14:'Geburtsdatum',
   15:'Beruf',
   16:'PersoNr',
   17:'PersoValidTo', 
   18:'Mitglied seit',
   19:'Mitglied bis' 
}

Bewertung_cols={
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

Buch_cols={
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

Exemplare_cols={
    1:'ID_Bestand',
    2:'Nr',
    3:'Kennung',
    4:'Zugriffsort',
    5:'Zustand'
}

Buch_cols={
    1:'ID_Autor',
    2:'Nr',
    3:'Titel',
    4:'Jahr',
    5:'Art',
}

Leihe_cols={
    0:'ID_Kunde',
    1:'ID_Exemplar',
    2:'Ausleihdatum',
    3:'Rueckgabedatum',
    4:'Verlaengerungsstatus',
    5:'Mahnstatus',
    6:'Fernleihe'
}

Mitgliedsstatus_cols={
    0:'ID_Mitgliedsstatus',
    2:'Bezeichnung',
    3:'Jahresbeitrag',
    4:'Mahnbetrag'
}

Beitragszahlung_cols={
    0:'ID_Zahlung',
    2:'ID_Kunde',
    3:'Datum',
    4:'Betrag',
}

In [32]:
dfs['Kunde']     = dfd['Neukunden'].reindex(columns=list(Kunde_cols.values()))
dfs['Bewertung'] = dfd['Bewertung'].reindex(columns=list(Bewertung_cols.values()))
dfs['Exemplare'] = df_bestand.reindex(columns=list(Exemplare_cols.values())).rename(columns={
                                                                        'ID_Bestand':'ID_Exemplar',
                                                                        'Nr':'ID_Buch'    
                                                                        })

dfs['Autor']   = df_bestand[['Autor', 'Herkunft']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'ID_Autor'})

dfs['Buch']    = pd.merge(df_bestand, 
                          dfs['Autor'], 
                          on=['Autor']
                         )[list(Buch_cols.values())]                                  \
                                                    .drop_duplicates()                \
                                                    .reset_index(drop=True)           \
                                                    .rename(columns={'Nr':'ID_Buch'})

dfs['Leihe'] = dfd['Transaktion'].reindex(columns=list(Leihe_cols.values())).reset_index().rename(columns={'index':'ID_Leihe'})


### more ###
dfs['Mitgliedsstatus'] = pd.DataFrame(columns=list(Mitgliedsstatus_cols.values()))
dfs['Beitragszahlung'] = dfd['Neukunden'].reindex(columns=list(Beitragszahlung_cols.values()))


In [95]:
dfs['Leihe'][['Ausleihdatum']]

dfd['Transaktion']

df = dfs['Leihe'].copy()
#df.loc[df["Aktion"] == "Leihe", "gender"] = 1

In [101]:

dfs['Leihe']['Ausleihdatum']=pd.to_datetime(dfs['Leihe']['Ausleihdatum'])
dfs['Leihe'].loc[dfd['Transaktion']['Aktion'] == "Leihe", "Ausleihdatum"] = dfd['Transaktion']['Datum']

dfs['Leihe']['Rueckgabedatum']=pd.to_datetime(dfs['Leihe']['Rueckgabedatum'])
dfs['Leihe'].loc[dfd['Transaktion']['Aktion'] == "Rückgabe", "Rueckgabedatum"] = dfd['Transaktion']['Datum']


In [102]:
dfs['Leihe']

,ID_Leihe,ID_Kunde,ID_Exemplar,Ausleihdatum,Rueckgabedatum,Verlaengerungsstatus,Mahnstatus,Fernleihe
0,0,750,98,NaT,2024-10-18 23:00:00,NaN,NaN,False
1,1,6282,139,2024-10-19 05:00:00,2024-10-19 05:00:00,NaN,NaN,False
2,2,5099,243,2024-10-19 07:00:00,2024-10-19 07:00:00,NaN,NaN,False
3,3,3066,101,NaT,2024-10-19 12:00:00,NaN,NaN,False
4,4,4066,224,2024-10-19 14:00:00,2024-10-19 14:00:00,NaN,NaN,False
5,5,6798,111,NaT,2024-10-19 19:00:00,NaN,NaN,False
6,6,424,70,2024-10-19 21:00:00,2024-10-19 21:00:00,NaN,NaN,False
7,7,245,235,NaT,2024-10-20 01:00:00,NaN,NaN,False
8,8,3715,167,2024-10-20 03:00:00,2024-10-20 03:00:00,NaN,NaN,False
9,9,3115,258,2024-10-20 05:00:00,2024-10-20 05:00:00,NaN,NaN,False


%6|1727353290.343|FAIL|rdkafka#consumer-1| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1727353290.556|FAIL|rdkafka#consumer-1| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY, 1 identical error(s) suppressed)
%6|1727353290.557|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1

%6|1727373654.407|FAIL|rdkafka#consumer-1| [thrd:GroupCoordinator]: GroupCoordinator: 127.0.0.1:9092: Disconnected (after 175203ms in state UP, 1 identical error(s) suppressed)
%6|1727373654.408|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1727373654.635|FAIL|rdkafka#consumer-1| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1727373654.652|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configurat

In [ ]:
df.columns

# Schicke die Tabellen an den SQL Server


In [37]:
from sqlalchemy import create_engine, inspect, text

USER = 'root'
PASSWORD = 'root'
HOST = 'localhost'          # see yml file
PORT = 3307
DATABASE = 'test_db_1'
 

def get_connection(database=DATABASE):
    return create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}"\
            .format(USER, PASSWORD, HOST, PORT, database)
    )

engine = get_connection('mysql')
with engine.connect() as connection:
    insp = inspect(engine)
    db_list = insp.get_schema_names()
    print(f"Connection to the {HOST} for user {USER} created successfully.")
    if DATABASE not in db_list:
        sql = text(f"CREATE DATABASE {DATABASE} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
        result = connection.execute(sql)
        if result:
            print(f"Database {DATABASE} created!")

Connection to the localhost for user root created successfully.


In [38]:
for Tabelle in dfs.keys():
    dfs[Tabelle].to_sql(Tabelle, con=engine,schema=DATABASE, index=False, if_exists='append') 

In [119]:


CDW_Tabellen_Liste = insp.get_table_names()

Statische_Tabellen = ['Exemplare', 'Autor', 'Buch']

for Tabelle in Statische_Tabellen:
    if Tabelle not in CDW_Tabellen_Liste:
        dfs[Tabelle].to_sql(Tabelle, con=engine,schema=DATABASE, index=False, if_exists='replace') 

for Tabelle in dfs.keys():
    if Tabelle not in Statische_Tabellen:
        dfs[Tabelle].to_sql(Tabelle, con=engine,schema=DATABASE, index=False, if_exists='append') 

Kunde
Bewertung
Leihe
Mitgliedsstatus
Beitragszahlung


%3|1727373845.031|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 19 identical error(s) suppressed)
%3|1727373868.033|FAIL|rdkafka#consumer-1| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Connect to ipv4#0.0.0.0:9092 failed: Connection refused (after 0ms in state CONNECT, 21 identical error(s) suppressed)
%3|1727373871.073|FAIL|rdkafka#consumer-1| [thrd:GroupCoordinator]: GroupCoordinator: 127.0.0.1:9092: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 3 identical error(s) suppressed)
%3|1727373875.033|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 16 identical error(s) suppressed)
%3|1727373903.034|FAIL|rdkafka#consumer-1| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Connect to ipv4#0.0.0.0:9092 failed: Connection r